# Callbacks
This notebook proposes an extension to the Problem solvers notebook.
We explore:
- The Callback concept
- The Progression Callback
- The Checkpoint Callback
- The CallbackList

## What is a callback?
A callback is an object that will execute specific actions that will be called at specific times. They serves as a way to alleviate the algorithm from unrelated actions, since they can take up several lines in the code. Examples of callbacks include:
- Following the progression in the console or with logs
- Making checkpoints of a model
- Saving results at each step
- Handling multiple break conditions in _for_ loops
- Etc.

In this setup, each of these action will be an individual callback, and each of these is responsible to fulfill its goal.

Let us explain further this concept with the help of an example. Suppose we want to implement an algorithm with 2 main parts. We want to execute code _not_ related to the algorithm itself in between. To achieve this without altering the algorithm conciseness, we define the following generic Callback object:

In [1]:
class Callback:
    """
    This is a generic Callback object that defines 4 methods to handle actions
    unrelated to the algorithm. 
    
    All callbacks should inherit from this one.
    """
    def before_part_1(self, *args, **kwargs):
        pass
    
    def after_part_1(self, *args, **kwargs):
        pass

    def before_part_2(self, *args, **kwargs):
        pass

    def after_part_2(self, *args, **kwargs):
        pass

The 4 methods of the Callback object will be called at the right time according to the name of the method. These methods take anything as arguments, since the callback could need anything to accomplish its goal.

The generic Callback object defines the 4 methods, even though they are not used and do nothing. This is so that inherited callbacks will have these methods well defined.

To give a more meaningful example, and to help us keep track of what is happening, define the these basic callbacks by inheriting from Callback: 

In [2]:
class BasicCallbackPart1(Callback):
    def before_part_1(self, *args, **kwargs):
        print('Doing something before part 1.')
    
    def after_part_1(self, *args, **kwargs):
        print('Doing something after part 1.')

        
class BasicCallbackPart2(Callback):
    def before_part_2(self, *args, **kwargs):
        print('Doing something before part 2.')
        
    def after_part_2(self, *args, **kwargs):
        print('Doing something after part 2.')

We use these callbacks inside the `ProblemSolver` class examined previously. We change the execution of the `solve` method.

In [3]:
class ProblemSolver:
    def solve(self, data_for_the_algo, callbacks_list):
        
        things_callbacks_need = (self,) # A reference to the current object is often needed by the callbacks
        
        # Before part 1
        for callback in callbacks_list: callback.before_part_1(things_callbacks_need)
        # Part 1
        self.solve_part_1(data_for_the_algo)
        # After part 1
        for callback in callbacks_list: callback.after_part_1(things_callbacks_need)
        
        # Maybe some line of codes
        
        # Before part 2
        for callback in callbacks_list: callback.before_part_2(things_callbacks_need)
        # Part 2
        self.solve_part_2(data_for_the_algo)
        # After part 2
        for callback in callbacks_list: callback.after_part_2(things_callbacks_need)
        
        return self
    
    def solve_part_1(self, data): pass
    
    def solve_part_2(self, data): pass

In [4]:
# Usage example
callbacks_list = [BasicCallbackPart1(), BasicCallbackPart2()]
problem_solver = ProblemSolver()
problem_solver.solve(1, callbacks_list)

Doing something before part 1.
Doing something after part 1.
Doing something before part 2.
Doing something after part 2.


This may seem rather involved to implement, compared to what we could have simply done inside the `solve` method. However, there are many advantages:
- Easy to maintain: You know exactly what do what and where in the code.
- No need for comment: all objects speak for themselves.
- Pass optional arguments to the constructor of the callback instead of the `solve` method.
- Clarity gain in the `solve` method: less lines and less optional arguments means easier to understand.

Let us look at some useful examples.

## The Progression Callback
We take a look at a basic Progression Callback. We can add some different verbose levels to adjust the number of outputs if we want.

In [5]:
class Progression(Callback):
    """
    This callback follows progression of the algorithm by printing at every steps.
    
    The verbose level are 0, 1 or 2.
    """
    def __init__(self, verbose_level=1):
        self.verbose_level = verbose_level
    
    def before_part_1(self, *args, **kwargs):
        print('Starting algorithm.')
    
    def after_part_1(self, *args, **kwargs):
        if self.verbose_level >= 1:
            print('Part 1 completed.')
        
    def before_part_2(self, *args, **kwargs):
        if self.verbose_level >= 2:
            print('We talk about part 2 of algorithm only if verbose is set high.')
        
    def after_part_2(self, *args, **kwargs):
        print('Finished algorithm')

In [6]:
# Usage example
callbacks_list = [Progression()]
problem_solver = ProblemSolver().solve(1, callbacks_list)

Starting algorithm.
Part 1 completed.
Finished algorithm


In [7]:
# Usage example
callbacks_list = [Progression(verbose_level=2)]
problem_solver = ProblemSolver().solve(1, callbacks_list)

Starting algorithm.
Part 1 completed.
We talk about part 2 of algorithm only if verbose is set high.
Finished algorithm


Another way of extending this callback would be to add an optional argument in the constructor to change the output from the terminal to a file for example.

## The Checkpoint Callback
Another useful action is to make a checkpoint of the model or the result after some steps to be able to retrieve it (at least partially) if something goes wrong. Here is an simple implementation.

In [8]:
import pickle as pkl

class Checkpoint(Callback):
    def __init__(self, filename, path='./', **pickling_kwargs):
        self.filename = filename
        self.path = path
        self.pickling_kwargs = pickling_kwargs
    
    def after_part_1(self, *args, **kwargs):
        model = self,
        self.save(model)
    
    def after_part_2(self, *args, **kwargs):
        model = self,
        self.save(model)
    
    def save(self, model):
        print(f'Saving model checkpoint to {self.path + self.filename}')
        with open(self.path + self.filename, 'wb') as file:
            pkl.dump(model, file, **self.pickling_kwargs)

In [9]:
# Usage example
callbacks_list = [Checkpoint(filename='test.pkl')]
problem_solver = ProblemSolver().solve(1, callbacks_list)

Saving model checkpoint to ./test.pkl
Saving model checkpoint to ./test.pkl


It goes without saying that this callback is more essential when the algorithm is an iterative long one.

As a last section, let us abstract callbacks one more level to simplify the code when there are many.

## The CallbackList
A CallbackList is an object which mimicks the behavior of a Callback and of a list at the same time.

In [10]:
class CallbackList:
    """
    This class mimicks the behavior of a Callback and of a list at the same time.
    """
    def __init__(self, callbacks_list):
        """
        Args:
            callbacks_list (iterable): An iterable of Callback objects.
        """
        self._callbacks_list = [callback for callback in callbacks_list]
    
    def append(self, item):
        self._callbacks_list.append(item)
    
    def __iter__(self):
        yield from self._callbacks_list

    def before_part_1(self, *args, **kwargs):
        for callback in self: callback.before_part_1(self, *args, **kwargs)
    
    def after_part_1(self, *args, **kwargs):
        for callback in self: callback.after_part_1(self, *args, **kwargs)

    def before_part_2(self, *args, **kwargs):
        for callback in self: callback.before_part_2(self, *args, **kwargs)

    def after_part_2(self, *args, **kwargs):
        for callback in self: callback.after_part_2(self, *args, **kwargs)
    

Now, the `ProblemSolver` code becomes

In [11]:
class ProblemSolver:
    def solve(self, data_for_the_algo, callbacks_list):
        
        things_callbacks_need = (self,) # A reference to the current object is often needed by the callbacks
        callbacks = CallbackList(callbacks_list)
        
        # Before part 1
        callbacks.before_part_1(things_callbacks_need)
        # Part 1
        self.solve_part_1(data_for_the_algo)
        # After part 1
        callbacks.after_part_1(things_callbacks_need)
        
        # Maybe some line of codes
        
        # Before part 2
        callbacks.before_part_2(things_callbacks_need)
        # Part 2
        self.solve_part_2(data_for_the_algo)
        # After part 2
        callbacks.after_part_2(things_callbacks_need)
        
        return self
    
    def solve_part_1(self, data): pass
    
    def solve_part_2(self, data): pass

In [12]:
# Usage example
callbacks_list = [Checkpoint(filename='test.pkl')]
callbacks_list.append(Progression())
problem_solver = ProblemSolver().solve(1, callbacks_list)

Starting algorithm.
Saving model checkpoint to ./test.pkl
Part 1 completed.
Saving model checkpoint to ./test.pkl
Finished algorithm
